# Scala School - Extending Examples

This is some of the code I wrote while following along with the [Scala School](http://twitter.github.io/scala_school/) tutorials.  I tried to combine multiple examples and extend them with other examples/things I already know about Scala.

## Table of Contents
- [Implement Generic map](#section1)
- [Implement Generic andThen and Compose](#section2)
- [Test my Map, andThen and Compose functions](#section3)
- [Case Class, Pattern Matching, Partial Functions, orElse](#section4)
- [Variance and Bounds](#section5)

<a id='section1'></a>

### Implement Generic map

In [86]:
def ourMap[T,K](numbers:Iterable[T], fn: T=>K)={
    //implementation of List's built in function map pass list and function
    //that outputs either the same or a different type and a list of
    //the function's output type will be returned since fn will be applied
    //to every element
    
    numbers.foldLeft(Iterable[K]()){
        (xs:Iterable[K],x:T) => xs++Iterable(fn(x))}
    }

defined function ourMap

<a id='section2'></a>

### Implement Generic andThen and Compose

In [78]:
def fandThenG[T,K,L](f:T => K,g: K=>L) = f andThen g
//generic implementation of composing functions using andthen

defined function fandThenG

In [79]:
def fComposeG[T,K,L](f:T => K,g: K=>L) = g compose f
//generic implementation of composing functions using compose

defined function fComposeG

<a id='section3'></a>

### Test my Map, andThen and Compose functions

In [80]:
val quinandThenString = fandThenG(fandThenG({x:Int => x*x},
                                            {y:Int => y*y*y}),
                                            {z:Int => z.toString})
//test function for fandThenG
//an Int will be squared, then cubed, then converted to a string

quinandThenString: Int => String = <function1>

In [81]:
val quinComposeString = fComposeG(fComposeG({x:Int => x*x}, 
                                            {y:Int => y*y*y}),
                                            {z:Int => z.toString})
//test function for fComposeG
//an Int will be squared, then cubed, then converted to a string

quinComposeString: Int => String = <function1>

In [87]:
ourMap(1.to(10), quinandThenString)
//test for ourMap and fandThenG
//each element of the list will be raised to the fifth and converted
//to a string

res69: Iterable[String] = List(
  "1",
  "64",
  "729",
  "4096",
  "15625",
  "46656",
  "117649",
  "262144",
  "531441",
  "1000000"
)

In [92]:
val arrayTest = 1.to(10).toArray
ourMap(arrayTest, quinComposeString)
//test for ourMap and fComposeG
//each element of the list will be raised to the fifth and converted
//to a string

arrayTest: Array[Int] = Array(
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10
)
res73_1: Iterable[String] = List(
  "1",
  "64",
  "729",
  "4096",
  "15625",
  "46656",
  "117649",
  "262144",
  "531441",
  "1000000"
)

<a id='section4'></a>

### Case Class, Pattern Matching, Partial Functions, orElse

In [42]:
case class Item(id:Long,action:String,stock:Int)//case class for an item
//id would contain a unique identifier for the item, action is what we
//would like performed on the item's stock and stock is the amount of 
//the item we have

defined class Item

In [48]:
val alreadyCompleted: PartialFunction[Item,Item] = {
    case i if i.action.contains("Completed_") =>
                                            println("Already Completed!")
                                            i
    
}
//Partial function checks if action already completed and if so returns new
//prints the action was completed and returns original instance of Item

alreadyCompleted: PartialFunction[Item, Item] = <function1>

In [47]:
val doubleStock: PartialFunction[Item,Item] = {
    case i if i.action=="double" => 
                                Item(i.id,"Completed_"+i.action, i.stock*2)
                                     }
//Partial function checks if wanted supply doubled and if so returns new
//instance of item declaring action completed and doubles stock

doubleStock: PartialFunction[Item, Item] = <function1>

In [67]:
val oneLessStock:PartialFunction[Item,Item]={
    case i if i.action=="oneLess" => i.stock match {
        case 0 => {
            println(s"${i.id} has a stock of 0!")
            i
        }
        case _ => Item(i.id,"Completed"+i.action,i.stock-1)
    } 
}
//Partial function checks if wanted one less and if so returns checks
//if stock is nonzero.  If the balance is zero an error is printed
//and the original Item instance is returned otherwise a new Item is
//returned where stock is decrimented by one

oneLessStock: PartialFunction[Item, Item] = <function1>

In [71]:
val wildCard: PartialFunction[Item, Item]={
    case i =>{
        println(s"${i.action} cannot be performed")
        i
    }
}
//Partial function that catches all remaining actions, prints an error 
//message and returns the instance of Item that was passed

wildCard: PartialFunction[Item, Item] = <function1>

In [72]:
val allActions = alreadyCompleted orElse doubleStock orElse 
                                            oneLessStock orElse wildCard 
//here I use orElse to compose my partial functions so that I now have
//one function to perform any of my actions

allActions: PartialFunction[Item, Item] = <function1>

In [60]:
allActions(Item(1,"Completed_double",1))//test alreadyCompleted

Already Completed!


res45: Item = Item(1L, "Completed_double", 1)

In [61]:
allActions(Item(1,"double",1))// test doubleStock

res46: Item = Item(1L, "Completed_double", 2)

In [65]:
allActions(Item(1,"oneLess",1))// test oneLess

res50: Item = Item(1L, "CompletedoneLess", 0)

In [69]:
allActions(Item(1,"oneLess",0))// test oneLess

1 has a stock of 0!


res54: Item = Item(1L, "oneLess", 0)

In [73]:
allActions(Item(1,"wildCard",0))// test wildCard

wildCard cannot be performed


res58: Item = Item(1L, "wildCard", 0)

<a id='section5'></a>

### Variance and Bounds

In [108]:
class Animal{val sound = "anything"}//create super class Animal
class Dog extends Animal{override val sound = "bark"}//create class Dog subclass of Animal
class Cat extends Animal{override val sound = "meow"}//create class Cat subclass of Animal
class FrenchBulldog extends Dog{override val sound = "squeak"}//create class FrenchBulldog which is a subclass of Dog which is a subclass of Animal
class Huskie extends Dog{override val sound = "howl"}//create class Huskie which is a subclass of Dog which is a subclass of Animal

defined class Animal
defined class Dog
defined class Cat
defined class FrenchBulldog
defined class Huskie

In [110]:
def getAnimalNoise[T <: Animal]:(T => String) = {a:Animal => a.sound}//return sound for any instance of a subclass of Animal

defined function getAnimalNoise

In [112]:
List(new FrenchBulldog, new Huskie,new Cat).map(getAnimalNoise)//test for getAnimalNoise function

res87: List[String] = List("squeak", "howl", "meow")

In [120]:
val frenchBulldogPuppy : (() => Dog) = () => new FrenchBulldog//create a new instance of FrenchBulldog that is of type Dog

frenchBulldogPuppy: () => Dog = <function0>

In [121]:
frenchBulldogPuppy() //test of frenchBulldogPuppy

res91: Dog = cmd84$$user$FrenchBulldog@23725a1